# <center>**`Project Details`**</center>

#### **Purpose**:

Tis project goal is matching a resume to a job description. A poorly aligned resume can lead to missed opportunities, even when the candidate is a strong fit. The project aims to showcase how we can make use of AI agents to help applicants tailor their resumes more strategically, uncover hidden gaps, and present a stronger case to recruiters — all with minimal effort.

A [Github repo](https://github.com/vikrambhat2/MultiAgents-with-CrewAI-ResumeJDMatcher) tackling this project exist and our goal will be to improve it by seperating the **Backend** and the **Frontend** logics.

##### **Why Split**?

 - The backend will hold business logic, agent orchestration, model calls, state, data processing, API endpoints, while the frontend focuses on the UI/UX, user interaction, session management, file upload, displaying results.
 - Scalability: Backend can scale independently of UI (and can even serve other clients)
 - Security: Sensitive logic, API keys, and resource-intensive processing are kept server-side
 - Performance: Streamlit remains snappy, while heavy lifting is offloaded to backend
 
##### **Responsibilities**

  - *<u>Backend</u>*: 
    - Expose REST API endpoints:
        - `/match`: Accepts resume + JD, returns match results and insights.
        - `/enhance`: Accepts resume + JD, returns resume improvement suggestions.
        - `/cover-letter`: Accepts resume + JD, returns a cover letter.
    - Agent orchestration: All CrewAI workflows run here.
    - Input validation, error handling.
    - PDF/text parsing if desired (or can also be handled in frontend, see below).
    - Optional: Authentication, user/session management, logging, monitoring.
    - Optional: Serve as an async queue for heavy jobs if latency is an issue (using Celery/RQ, etc.).
 
 - *<u>Frontend</u>(Streamlit)*
    - UI for uploading files, entering/pasting text.
    - Visualization: Render reports, scores, enhanced resume, cover letter, etc.
    - API client: Handles all interaction with FastAPI backend.
    - Light preprocessing: E.g., local PDF parsing if you want to send plain text to backend (saves bandwidth).
    - Session/user state, feedback, download links, etc.

Here is how the system works (Flow):

 1. User uploads resume & JD (PDF or text) → Streamlit UI

 2. Frontend extracts or passes files → Sends to FastAPI (as text or file)

 3. FastAPI endpoint receives, orchestrates CrewAI agents, returns structured results

 4. Streamlit displays results, progress, suggestions, etc.

#### **Constraints**:

 - None


#### **Tools**:

 - Use local **ollama** model

#### **Requirements**:
 - Make it work as expected


***

## <center>**`Implementation`**</center>

## **`Backend`**

### Core

#### PDF Parsing

In [38]:
%%writefile ../backend/app/core/pdf_parser.py

#backend/app/core/pdf_parser.py
from typing import Union
from pathlib import Path
from PyPDF2 import PdfReader


class PDFParser:
    """Handles PDF and plain text extraction."""

    def extract_text(self, file: Union[Path, bytes]) -> str:
        if isinstance(file, Path):
            with open(file, "rb") as f:
                reader = PdfReader(f)
                return self._extract_all(reader)
        elif isinstance(file, bytes):
            from io import BytesIO
            reader = PdfReader(BytesIO(file))
            return self._extract_all(reader)
        else:
            raise ValueError("Unsupported file type for PDFParser.")
        
    def _extract_all(self, reader: PdfReader) -> str:
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
        return text.strip()

Writing ../backend/app/core/pdf_parser.py


#### Agent Orchestrator

In [48]:
%%writefile ../backend/app/core/agent_orchestrator.py

# backend/app/core/agent_orchestrator.py
from typing import Dict
import time

class AgentOrchestrator:
    """Handles agent pipeline for resume-JD matching."""
    def __init__(self):
        # We will init agents here
        pass

    def run(self, job_type: str, data: Dict) -> Dict:
        """Executes the specified agent pipeline.
        Args:
            job_type: 'match', 'enhance', or 'cover_letter'
            data: Dict with 'resume' and 'jd' (plain text)
        Returns:
            Dict with results
        """
        time.sleep(2)
        if job_type == "match":
            return {"status": "done", "result": {"match_score": 85, "insights": "Strong match for key requirements."}}
        elif job_type == "enhance":
            return {"status": "done", "result": {"improvements": "Add more technical keywords from JD."}}
        elif job_type == "cover_letter":
            return {"status": "done", "result": {"cover_letter": "Dear Hiring Manager, ..."}}
        else:
            return {"status": "error", "result": {"error": "Invalid job_type"}}

Writing ../backend/app/core/agent_orchestrator.py


### Job Queueing + Celery for distributed background job handling

#### Queueing

In [50]:
%%writefile ../backend/app/core/async_queue.py
# backend/app/core/async_queue.py
from backend.app.core.tasks import run_agent_job

class AsyncJobQueueCelery:
    """Async job queue using Celery."""
    def submit_job(self, job_type: str, payload: dict) -> str:
        celery_result = run_agent_job.delay(job_type, payload)
        return celery_result.id
    
    def get_status(self, job_id: str) -> dict:
        from celery.result import AsyncResult
        from backend.worker.worker import celery_app
        result = AsyncResult(job_id, app=celery_app)
        status = result.status
        value = result.result if result.successful() else None
        return {"status": status, "result": value}
    
# Singleton
queue = AsyncJobQueueCelery()

Overwriting ../backend/app/core/async_queue.py


#### Celery config

In [54]:
%%writefile ../backend/celeryconfig.py
# backend/celeryconfig.py

# redis is in another docker container
# if it's not the case for you,
# use : "redis://localhost:6379/0"

broker_url = "redis://host.docker.internal:6379/0"   
result_backend = "redis://host.docker.internal:6379/0"
task_serializer = "json"
result_serializer = "json"
accept_content = ["json"]
timezone = "UTC"
enable_utc = True

Overwriting ../backend/celeryconfig.py


#### Celery worker

In [55]:
%%writefile ../backend/worker/worker.py
# backend/worker/worker.py

from celery import Celery

celery_app = Celery("resume_jd_matcher")
celery_app.config_from_object("backend.celeryconfig")

import backend.app.core.tasks  # Ensure all tasks are registered

Overwriting ../backend/worker/worker.py


#### Celery Task

In [49]:
%%writefile ../backend/app/core/tasks.py
# backend/app/core/tasks.py

from backend.worker.worker import celery_app
from backend.app.core.agent_orchestrator import AgentOrchestrator

@celery_app.task(name="run_agent_job")
def run_agent_job(job_type: str, data: dict):
    orchestrator = AgentOrchestrator()
    result = orchestrator.run(job_type, data)
    return result

Writing ../backend/app/core/tasks.py


### Backend api

#### Data models

In [52]:
%%writefile ../backend/app/models/job_models.py

#backend/app/models/job_models.py

from pydantic import BaseModel
from typing import Optional, Dict

class ResumeJDRequest(BaseModel):
    resume: Optional[str] = None
    jd: Optional[str] = None
    job_type: str

class PDFUploadResponse(BaseModel):
    extracted_text: str

class JobSubmitResponse(BaseModel):
    job_id: str

class JobStatusResponse(BaseModel):
    status: str
    result: Optional[Dict] = None

Overwriting ../backend/app/models/job_models.py


#### Router

In [53]:
%%writefile ../backend/app/api/routes.py
#backend/app/api/routes.py

from fastapi import APIRouter, File, UploadFile
from backend.app.core.pdf_parser import PDFParser
from backend.app.core.async_queue import queue
from backend.app.models.job_models import(
    ResumeJDRequest,
    PDFUploadResponse,
    JobSubmitResponse,
    JobStatusResponse
)


api_router = APIRouter()

@api_router.get("/health", tags=["Health"])
def health_check():
    return {"status": "ok"}

@api_router.post("/parse-pdf", response_model=PDFUploadResponse, tags=["Parsing"])
async def parse_pdf_endpoint(file: UploadFile = File(...)):
    """Extract text from uploaded PDF file."""
    content = await file.read()
    parser = PDFParser()
    try:
        text = parser.extract_text(content)
        return PDFUploadResponse(extracted_text=text)
    except Exception as e:
        return PDFUploadResponse(extracted_text=f"Error: {str(e)}")

@api_router.post("/submit-job", response_model=JobSubmitResponse, tags=["Jobs"])
async def submit_job(request: ResumeJDRequest):
    """Submit a matching/enhancing/cover letter job."""
    job_id = queue.submit_job(request.job_type, request.dict())
    return JobSubmitResponse(job_id=job_id)

@api_router.get("/job-status/{job_id}", response_model=JobStatusResponse, tags=["Jobs"])
async def job_status(job_id: str):
    status = queue.get_status(job_id)
    return JobStatusResponse(**status)

Overwriting ../backend/app/api/routes.py


#### App

In [37]:
%%writefile ../backend/app/main.py
#backend/app/main.py

from fastapi import FastAPI
from backend.app.api.routes import api_router

class JDMatcherApp:
    def __init__(self):
        self.app = FastAPI(
            title="Resume-JD Matcher API",
            description="Backend for matching candidate resumes to job descriptions using AI agents.",
            version="0.1.0"
        )
        self.include_routers()


    def include_routers(self):
        self.app.include_router(api_router)

def get_app():
    """Entrypoint for ASGI"""
    return JDMatcherApp().app

# Run with 'uvicorn backend.app.main:get_app'
app = get_app()

Overwriting ../backend/app/main.py


## **`Frontend`**